# RDFIA TME 9 : Visualization of Neural Networks


In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import random
import math

import torch
from torch.nn import functional as F
import torchvision
import torchvision.transforms as T
import numpy as np
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from PIL import Image

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['savefig.dpi'] = 150

## Loading the model

In [ ]:
from utils import *

# Set URL for pretrained model SqueezeNet (Iandola et al (2016))
# This is the SqueezeNet network from Iandola et al (2016) trained on ImageNet,
# it achieve comparable results to AlexNet on ImageNet while being very compact
torchvision.models.vgg.model_urls[
    "squeezenet1_1"] = "http://webia.lip6.fr/~douillard/rdfia/squeezenet1_1-f364aa15.pth"
os.environ["TORCH_HOME"] = "./pytorch_models"

# loading the model
model = torchvision.models.squeezenet1_1(pretrained=True)
# put in in test mode
model.eval()
# freeze the paramaters
for param in model.parameters():
    param.requires_grad = False

## Loading example images

In [ ]:
# Load example images: 25 images from ImageNet
f = np.load("data/imagenet_val_25.npz", allow_pickle=True)
data, target, class_names = f["X"], f["y"], f["label_map"].item()
class_names = {k: v.split(',')[0] for (k, v) in class_names.items()}

def show_images(savepath=None):
    plt.figure(figsize=(15, 7))
    for i in range(24):
        plt.subplot(4, 6, i + 1)
        plt.imshow(data[i])
        plt.title(class_names[target[i]])
        plt.axis('off')
    plt.gcf().tight_layout()
    if savepath: plt.savefig(savepath)
    plt.show()

show_images(savepath='figures/Images.jpeg')

## Saliency maps

In [ ]:
from saliency_maps import compute_saliency_maps

def show_saliency_maps(data, target, model, class_names, savepath=None):
    # convert data and target from numpy arrays to Torch Tensors
    data_tensor = torch.cat([preprocess(Image.fromarray(x)) for x in data], dim=0)
    target_tensor = torch.LongTensor(target)

    # compute saliency maps for images in X
    saliency = compute_saliency_maps(data_tensor, target_tensor, model)
    # convert the saliency map from torch.Tensor to numpy.array and show images
    # and saliency maps together.
    saliency = saliency.numpy()
    N = data.shape[0]
    for i in range(N):
        plt.subplot(2, N, i + 1)
        plt.imshow(data[i])
        plt.axis('off')
        plt.title(class_names[target[i]])
        plt.subplot(2, N, N + i + 1)
        plt.imshow(saliency[i], cmap=plt.cm.hot)
        plt.axis('off')
        plt.gcf().set_size_inches(12, 5)
    
    plt.gcf().tight_layout()
    if savepath: plt.savefig(savepath, bbox_inches='tight')
    plt.show()

for i in range(5):
    show_saliency_maps(data[5*i: 5*i + 5], target[5*i: 5*i + 5], model, class_names,
                       savepath='./figures/saliency-maps/batch_{:02}'.format(i))

## Fooling images

### Optimizing the output for the target class

In [ ]:
from fooling_examples import make_fooling_image

x_tensor = preprocess(Image.fromarray(data[0]))
dest_y = 6
x_foooled, t = make_fooling_image(x_tensor, dest_y, model, max_iters=100, confidence=0.99, plot_progress=True)
print('Number of iterations:')

In [ ]:
def show_fooling_images(data, target, destination_y, model, class_names, optimization='class_output', savedir=None):

    data_tensor = torch.cat([preprocess(Image.fromarray(x)) for x in data], dim=0)
    dest_y = destination_y
    for idx in range(0, len(target)):
        # prepare tensor data and its fooling version
        x_tensor = data_tensor[idx][None]
        y = target[idx]
        # original prediction score for the class of x
        # (not necessarily the predicted label!)
        y_score_before = F.softmax(model(x_tensor), 1)[0, y].item()

        x_fooling, t = make_fooling_image(x_tensor, destination_y, model, max_iters=100, optimization=optimization)
        # verify the predicted class on the fooling example
        scores = model(x_fooling)
        dest_y_score, pred_y = map(lambda t: t.item(), F.softmax(scores, 1).max(1))
        success = (dest_y == pred_y)
        
        # Plots
        x_fooling_np = deprocess(x_fooling.clone())
        x_fooling_np = np.asarray(x_fooling_np).astype(np.uint8)

        plt.subplot(1, 4, 1)
        plt.imshow(data[idx])
        plt.title("Real: {}\nConfidence: {:.2%}".format(class_names[y], y_score_before))
        plt.axis('off')

        plt.subplot(1, 4, 2)
        plt.imshow(x_fooling_np)
        plt.title("{}: {}\n{} \n{} iterations"
                  .format('Fooling' if success else 'Failed to fool', class_names[dest_y],
                          'Confidence: {:.2%}'.format(dest_y_score) if success else '', t))
        plt.axis('off')

        plt.subplot(1, 4, 3)
        x_pre = preprocess(Image.fromarray(data[idx]))
        diff = np.asarray(deprocess(x_fooling - x_pre, should_rescale=False))
        plt.imshow(diff)
        plt.title('Difference')
        plt.axis('off')

        plt.subplot(1, 4, 4)
        diff = np.asarray(deprocess(10 * (x_fooling - x_pre), should_rescale=False))
        plt.imshow(diff)
        plt.title('Magnified difference (10x)')
        plt.axis('off')

        plt.gcf().set_size_inches(12, 5)
        plt.gcf().tight_layout()
        if savedir:
            plt.savefig(Path(savedir)/'{:02}_{}.jpg'.format(idx, class_names[y]), bbox_inches='tight')
        plt.show()

savedir = "./figures/fooling-examples/optimization='class_output'"
show_fooling_images(data, target, 6, model, class_names, optimization='class_output', savedir=savedir)

## Class visualization

### Visualizing several ImageNet classes

In [ ]:
# selected classes
from class_visualization import create_class_visualization

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor # uncomment this to use GPU

target_selection  = [  8,    # Hen
                     281,  # Tabby cat
                     187,  # Yorkshire Terrier
                      76,   # Tarantula
                      78,   # Tick
                     683,  # Oboe
                     366,  # Gorilla
                     604,  # Hourglass
                     130,  # Flamingo
                   ]
# start with given arbitrary parameters
# args = dict(
#     init_img = None,
#     l2_reg = 1e-3,
#     learning_rate =  5,
#     num_iterations = 200,
#     blur_every = 10,
#     blur_width = 0.5,
#     max_jitter = 16,
#     clamp=True,
#     show_every = 25)
args = dict(
    init_img = None,
    l2_reg = 1e-4,
    learning_rate =  5.0,
    num_iterations = 200,
    blur_every = 10,
    blur_width = 0.5,
    max_jitter = 16,
    clamp=True,
    show_every = 25)

ignore_keys = {'show_every', 'class_names'}
# get hyperparameters with values in a dict
hparams = {key.replace('_','-'): val for key, val in args.items() if key not in ignore_keys}
# generate a name for the experiment
expe_name = '_'.join(f"{key}={val}" for key, val in hparams.items())

print('Experiment name:', expe_name)
(Path('./figures/class-visualizations/')/expe_name).mkdir(parents=True, exist_ok=True)

class_images = []  # put all images in a list for plotting them together
for target_y in target_selection:
    class_name = class_names[target_y]
    savepath = './figures/class-visualizations/{}/{}.jpg' .format(expe_name, class_name)
    out = create_class_visualization(target_y, model, dtype, **args, class_name=class_name, savepath=savepath)
    class_images.append(out)

In [ ]:
# # save the generated PIL images into a file
# import pickle
# savepath = Path('./data/class-visualizations/')/expe_name
# savepath.mkdir(parents=True, exist_ok=True)
# with open(savepath/'images.pkl', 'wb') as fp:
#     pickle.dump(class_images, fp)

# # with open(savepath/'images.pkl', 'rb') as fp:
# #     class_images = pickle.load(fp)

In [ ]:
def show_images(images, target, class_names, savepath=None, cols=3, figsize=(10, 10)):
    """Show a list of images in a grid layout."""
    rows = math.ceil(len(images) / cols)

    plt.figure(figsize=figsize)
    for i in range(len(images)):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i])
        class_name = class_names[target[i]]
        plt.title(class_name)
        plt.axis('off')
    plt.gcf().tight_layout()

    if savepath: plt.savefig(savepath, bbox_inches='tight')   
    plt.show()

savepath = './figures/class-visualizations/{}/Images.jpg' .format(expe_name)
show_images(class_images, target_selection, class_names, savepath=savepath)

### Varing the number of iterations

In [ ]:
# selected classes
target_y = 130 # Flamingo

# start with given arbitrary parameters
args = dict(
    init_img = None,
    l2_reg = 1e-3,
    learning_rate =  5,
    blur_every = 10,
    blur_width = 0.5,
    max_jitter = 16,
    clamp=True)

ignore_keys = {'show_every', 'class_names'}
# get hyperparameters with values in a dict
hparams = {key.replace('_','-'): val for key, val in args.items() if key not in ignore_keys}
# generate a name for the experiment
expe_name = '_'.join(f"{key}={val}" for key, val in hparams.items())

print('Experiment name:', expe_name)
savedir = Path('./figures/class-visualizations/')/expe_name
savedir.mkdir(parents=True, exist_ok=True)


num_iterations_values = [50, 100, 150, 200, 300, 400, 600, 800]
class_images = []
for num_iterations in num_iterations_values:
    args.update(num_iterations = num_iterations,
                show_every = num_iterations)
    
    print('Num iterations:', num_iterations)
    class_name = class_names[target_y]
    savepath = savedir/'{}__num_iterations={}.jpg'.format(class_name, num_iterations)
    out = create_class_visualization(target_y, model, dtype, **args, class_name=class_name, savepath=savepath)
    class_images.append(out)

In [ ]:
def show_images(images, class_name, savepath=None, cols=4, figsize=(12, 7.2)):
    """Show a list of images in a grid layout."""
    rows = math.ceil(len(images) / cols)

    plt.figure(figsize=figsize)
    for i, num_iterations in zip(range(len(images)), num_iterations_values):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i])
        plt.title("{} \nNum iterations: {}".format(class_name, num_iterations))
        plt.axis('off')
    plt.gcf().tight_layout()

    if savepath: plt.savefig(savepath, bbox_inches='tight')   
    plt.show()

print(savepath)
savepath = savedir/'{}__images.jpg'.format(class_name)
show_images(class_images, class_name, savepath=savepath)

### Varying the L2 reg. coefficient

In [ ]:
# selected classes
target_y = 130 # Flamingo

# start with given arbitrary parameters
args = dict(
    init_img = None,
    learning_rate =  5,
    num_iterations = 200,
    blur_every = 10,
    blur_width = 0.5,
    max_jitter = 16,
    clamp=True,
    show_every = 200)
  
ignore_keys = {'show_every', 'class_names'}
# get hyperparameters with values in a dict
hparams = {key.replace('_','-'): val for key, val in args.items() if key not in ignore_keys}
# generate a name for the experiment
expe_name = '_'.join(f"{key}={val}" for key, val in hparams.items())

print('Experiment name:', expe_name)
savedir = Path('./figures/class-visualizations/')/expe_name
savedir.mkdir(parents=True, exist_ok=True)

l2_reg_values = [0., 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1.0, 5.0, 10.0]
class_images = []
for l2_reg in l2_reg_values:
    args.update(l2_reg = l2_reg)
    print('L2 reg:', l2_reg)
    class_name = class_names[target_y]
    savepath = savedir/'{}__l2_reg={}.jpg'.format(class_name, l2_reg)
    out = create_class_visualization(target_y, model, dtype, **args, class_name=class_name, savepath=savepath)
    class_images.append(out)

In [ ]:

def show_images(images, class_name, savepath=None, cols=4, figsize=(12, 10.5)):
    """Show a list of images in a grid layout."""
    rows = math.ceil(len(images) / cols)
    plt.figure(figsize=figsize)
    for i, l2_reg in zip(range(len(images)), l2_reg_values):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i])
        plt.title("{} \nL2 coefficient: {}".format(class_name, l2_reg))
        plt.axis('off')
    plt.gcf().tight_layout()
    
    if savepath: plt.savefig(savepath, bbox_inches='tight')   
    plt.show()

savepath = savedir/'{}__images.jpg'.format(class_name)
show_images(class_images, class_name, savepath=savepath)

### Starting from an ImageNet image

Here we use the ImageNet images as initial images instead of random images. The target class is set as the label of each image.

In [ ]:

args = dict(
    init_img = 'sample',
    l2_reg = 1e-4,
    learning_rate =  1,
    num_iterations = 50,
    blur_every = 10,
    blur_width = 0.5,
    max_jitter = 16,
    clamp=True,
    show_every = 15)

ignore_keys = {'show_every'}
# get hyperparameters with values in a dict
hparams = {key.replace('_','-'): val for key, val in args.items() if key not in ignore_keys}
# generate a name for the experiment
expe_name = '_'.join(f"{key}={val}" for key, val in hparams.items())
print('Experiment name:', expe_name)

savedir = Path('./figures/class-visualizations/')/expe_name
savedir.mkdir(parents=True, exist_ok=True)

class_images = []
for i in range(len(target)):
    target_y = target[i]
    init_img = torch.Tensor(preprocess(Image.fromarray(data[i])))
    class_name = class_names[target_y]

    args.update(init_img=init_img)
    savepath = savedir/'{:02}_{}'.format(idx, class_name)
    out = create_class_visualization(target_y, model, dtype, **args, class_name=class_name, savepath=savepath)
    class_images.append(out)

In [ ]:
import math

images_selection, target_selection = \
    list(zip(*[(class_images[i], target[i]) for i in [3, 11, 13, 24]]))


def show_images(images, target, class_names, savepath=None, cols=4, figsize=(10, 4)):
    """Show a list of images in a grid layout."""
    rows = math.ceil(len(images) / cols)
    
    plt.figure(figsize=figsize)
    for i in range(len(images)):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i])
        class_name = class_names[target[i]]
        plt.title(class_name)
        plt.axis('off')
    plt.gcf().tight_layout()

    if savepath: plt.savefig(savepath, bbox_inches='tight')   
    plt.show()

savepath = './figures/class-visualizations/{}/Images.jpg' .format(expe_name)
show_images(images_selection, target_selection, class_names, savepath=savepath)